### Import

In [35]:
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function
#from sklearn.datasets import fetch_20newsgroups

### Load Data

In [17]:
categories = ['alt.atheism', 'soc.religion.christian']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
class_names = ['atheism', 'christian']

In [33]:
#newsgroups_train['target'][1]
print(newsgroups_train['DESCR'])
print('-----')
newsgroups_train['data'][0]
#newsgroups_train.keys()

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality 

'From: nigel.allen@canrem.com (Nigel Allen)\nSubject: library of congress to host dead sea scroll symposium april 21-22\nLines: 96\n\n\n Library of Congress to Host Dead Sea Scroll Symposium April 21-22\n To: National and Assignment desks, Daybook Editor\n Contact: John Sullivan, 202-707-9216, or Lucy Suddreth, 202-707-9191\n          both of the Library of Congress\n\n   WASHINGTON, April 19  -- A symposium on the Dead Sea \nScrolls will be held at the Library of Congress on Wednesday,\nApril 21, and Thursday, April 22.  The two-day program, cosponsored\nby the library and Baltimore Hebrew University, with additional\nsupport from the Project Judaica Foundation, will be held in the\nlibrary\'s Mumford Room, sixth floor, Madison Building.\n   Seating is limited, and admission to any session of the symposium\nmust be requested in writing (see Note A).\n   The symposium will be held one week before the public opening of a\nmajor exhibition, "Scrolls from the Dead Sea: The Ancient Library

In [ ]:
# vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
# train_vectors = vectorizer.fit_transform(newsgroups_train.data)
# test_vectors = vectorizer.transform(newsgroups_test.data)
# Now, let's say we want to use random forests for classification. It's usually hard to understand what random forests are doing, especially with many trees.

# rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
# rf.fit(train_vectors, newsgroups_train.target)
# RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#             max_depth=None, max_features='auto', max_leaf_nodes=None,
#             min_samples_leaf=1, min_samples_split=2,
#             min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
#             oob_score=False, random_state=None, verbose=0,
#             warm_start=False)
# pred = rf.predict(test_vectors)
# sklearn.metrics.f1_score(newsgroups_test.target, pred, average='binary')
# 0.92093023255813955
# We see that this classifier achieves a very high F score. The sklearn guide to 20 newsgroups indicates that Multinomial Naive Bayes overfits this dataset by learning irrelevant stuff, such as headers. Let's see if random forests do the same.

# Explaining predictions using lime
# Lime explainers assume that classifiers act on raw text, but sklearn classifiers act on vectorized representation of texts. For this purpose, we use sklearn's pipeline, and implements predict_proba on raw_text lists.

# from lime import lime_text
# from sklearn.pipeline import make_pipeline
# c = make_pipeline(vectorizer, rf)
# print(c.predict_proba([newsgroups_test.data[0]]))
# [[ 0.274  0.726]]
# Now we create an explainer object. We pass the class_names a an argument for prettier display.

# from lime.lime_text import LimeTextExplainer
# explainer = LimeTextExplainer(class_names=class_names)
# We then generate an explanation with at most 6 features for an arbitrary document in the test set.

# idx = 83
# exp = explainer.explain_instance(newsgroups_test.data[idx], c.predict_proba, num_features=6)
# print('Document id: %d' % idx)
# print('Probability(christian) =', c.predict_proba([newsgroups_test.data[idx]])[0,1])
# print('True class: %s' % class_names[newsgroups_test.target[idx]])
# Document id: 83
# Probability(christian) = 0.414
# True class: atheism
# The classifier got this example right (it predicted atheism).
# The explanation is presented below as a list of weighted features.

# exp.as_list()
# [(u'Posting', -0.15748303818990594),
#  (u'Host', -0.13220892468795911),
#  (u'NNTP', -0.097422972255878093),
#  (u'edu', -0.051080418945152584),
#  (u'have', -0.010616558305370854),
#  (u'There', -0.0099743822272458232)]
# These weighted features are a linear model, which approximates the behaviour of the random forest classifier in the vicinity of the test example. Roughly, if we remove 'Posting' and 'Host' from the document , the prediction should move towards the opposite class (Christianity) by about 0.27 (the sum of the weights for both features). Let's see if this is the case.

# print('Original prediction:', rf.predict_proba(test_vectors[idx])[0,1])
# tmp = test_vectors[idx].copy()
# tmp[0,vectorizer.vocabulary_['Posting']] = 0
# tmp[0,vectorizer.vocabulary_['Host']] = 0
# print('Prediction removing some features:', rf.predict_proba(tmp)[0,1])
# print('Difference:', rf.predict_proba(tmp)[0,1] - rf.predict_proba(test_vectors[idx])[0,1])
# Original prediction: 0.414
# Prediction removing some features: 0.684
# Difference: 0.27
# Pretty close!
# The words that explain the model around this document seem very arbitrary - not much to do with either Christianity or Atheism.
# In fact, these are words that appear in the email headers (you will see this clearly soon), which make distinguishing between the classes much easier.

# Visualizing explanations
# The explanations can be returned as a matplotlib barplot:

# %matplotlib inline
# fig = exp.as_pyplot_figure()

# The explanations can also be exported as an html page (which we can render here in this notebook), using D3.js to render graphs.

# exp.show_in_notebook(text=False)
# Prediction probabilities
# 0.59
# atheism
# 0.41
# christian
# atheism
# christian
# Posting
# 0.16
# Host
# 0.13
# NNTP
# 0.10
# edu
# 0.05
# have
# 0.01
# There
# 0.01
# Alternatively, we can save the fully contained html page to a file:

# exp.save_to_file('/tmp/oi.html')
# Finally, we can also include a visualization of the original document, with the words in the explanations highlighted. Notice how the words that affect the classifier the most are all in the email header.

# exp.show_in_notebook(text=True)
# Prediction probabilities
# 0.59
# atheism
# 0.41
# christian
# atheism
# christian
# Posting
# 0.16
# Host
# 0.13
# NNTP
# 0.10
# edu
# 0.05
# have
# 0.01
# There
# 0.01
# Text with highlighted words
# From: johnchad@triton.unm.edu (jchadwic)
# Subject: Another request for Darwin Fish
# Organization: University of New Mexico, Albuquerque
# Lines: 11
# NNTP-Posting-Host: triton.unm.edu

# Hello Gang,

# There have been some notes recently asking where to obtain the DARWIN fish.
# This is the same question I have and I have not seen an answer on the
# net. If anyone has a contact please post on the net or email me.

# Thanks,

# john chadwick
# johnchad@triton.unm.edu
# or
# That's it for this tutorial. Random forests were just an example, this explainer works for any classifier you may want to use, as long as it implements predict_proba.